# Phase 9 — IFRS 9 : PD → ECL → KPI portefeuille

## Objectif
Construire une chaîne complète et reproductible :
1) Créer la cible défaut (default_flag)
2) Préparer les features (nettoyage/encodage)
3) Entraîner un modèle PD (régression logistique)
4) Calculer EAD / LGD / ECL (IFRS 9)
5) Produire KPI portefeuille + exports CSV pour Tableau (Phase 10)

Formule IFRS 9 :
**ECL = PD × LGD × EAD**


In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [12]:
# 1) Portefeuille crédits (LendingClub "light")
df = pd.read_csv("data/interim/lendingclub_light.csv", low_memory=False)

# 2) niveau de risque climat par État
climate = pd.read_csv("../reports/tableau_exports/07_climate_risk_by_state.csv")

print(loans.shape)
loans.head(2)


(250000, 8)


,loan_amnt,term,int_rate,grade,annual_inc,loan_status,addr_state,dti
0,20000.0,36 months,13.99,C,65000.0,Fully Paid,CA,13.68
1,7000.0,36 months,9.16,B,35000.0,Fully Paid,TX,22.39


In [13]:
# Définition de la variable de défaut
df["default_flag"] = df["loan_status"].astype(str).str.lower().apply(
    lambda x: 1 if x in ["charged off", "default"] else 0
)

df["default_flag"].value_counts(normalize=True)


default_flag
0    0.800412
1    0.199588
Name: proportion, dtype: float64

In [15]:
# Transformation de la durée du prêt : "36 months" → 36
df["term_num"] = df["term"].str.extract("(\d+)").astype(int)

df[["term", "term_num"]].head()


,term,term_num
0,36 months,36
1,36 months,36
2,36 months,36
3,36 months,36
4,36 months,36


In [22]:
grade_mapping = {"A":1,"B":2,"C":3,"D":4,"E":5,"F":6,"G":7}
df["grade_num"] = df["grade"].astype(str).map(grade_mapping)

df[["grade","grade_num"]].head()


,grade,grade_num
0,C,3
1,B,2
2,B,2
3,C,3
4,B,2


In [23]:
features = ["loan_amnt", "term_num", "int_rate", "grade_num", "annual_inc", "dti"]

X = df[features]
y = df["default_flag"]

# diagnostic NaN (juste pour info)
X.isna().sum().sort_values(ascending=False)


dti           81
loan_amnt      0
term_num       0
int_rate       0
grade_num      0
annual_inc     0
dtype: int64

In [18]:
X.dtypes


loan_amnt     float64
term_num        int64
int_rate      float64
grade_num       int64
annual_inc    float64
dti           float64
dtype: object

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),   # remplace NaN par médiane
    ("scaler", StandardScaler()),                    # standardise
    ("model", LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=1000))
])

pipe.fit(X_train, y_train)


Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler()),
                ('model', LogisticRegression(max_iter=1000))])

In [25]:
y_proba_test = pipe.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_proba_test)
auc


np.float64(0.7003301145443586)

In [27]:
df["pd_pred"] = pipe.predict_proba(X)[:, 1]
df[["pd_pred"]].head()


,pd_pred
0,0.183604
1,0.131300
2,0.131330
3,0.222908
4,0.129272


## IFRS 9 — Paramètres

- **PD** : `pd_pred` (probabilité de défaut) issue du modèle
- **EAD** : exposition au défaut (approche simple : montant du prêt)
- **LGD** : perte en cas de défaut (approche simple : valeur fixe)

On applique ensuite :
**ECL = PD × LGD × EAD**


In [30]:
LGD = 0.60
df["EAD"] = df["loan_amnt"]

df["ECL"] = df["pd_pred"] * LGD * df["EAD"]

df[["pd_pred", "EAD", "ECL"]].head()


,pd_pred,EAD,ECL
0,0.183604,20000.0,2203.252744
1,0.131300,7000.0,551.460036
2,0.131330,20000.0,1575.957618
3,0.222908,16000.0,2139.913680
4,0.129272,4000.0,310.253096


In [33]:
kpi_portfolio = pd.DataFrame([{
   
    "total_ead": df["EAD"].sum(),
    "avg_pd": df["pd_pred"].mean(),
    "lgd_assumption": LGD,
    "total_ecl": df["ECL"].sum()
}])

kpi_portfolio


,total_ead,avg_pd,lgd_assumption,total_ecl
0,3.601679e+09,0.199742,0.6,4.674403e+08


In [34]:
export_dir = "../reports/tableau_exports"

df[["pd_pred", "EAD", "ECL"]].to_csv(
    f"{export_dir}/09_ecl_per_loan.csv", index=False
)

kpi_portfolio.to_csv(
    f"{export_dir}/09_portfolio_kpi.csv", index=False
)

print("Exports OK:",
      f"{export_dir}/09_ecl_per_loan.csv",
      f"{export_dir}/09_portfolio_kpi.csv")


Exports OK: ../reports/tableau_exports/09_ecl_per_loan.csv ../reports/tableau_exports/09_portfolio_kpi.csv


## Synthèse 

- Construction d’une **PD** via régression logistique (pipeline robuste : imputation + scaling)
- Application du cadre IFRS 9 avec **EAD/LGD** explicites
- Production des livrables :
  - ECL par prêt
  - KPI portefeuille (pilotage managérial)
- Base prête pour la Phase 10 (Tableau Public).
